In [2]:
import os
os.getcwd()

import sys 
sys.path.append(os.path.abspath(os.path.join('../')))
from langchain import LLMChain
from langchain import SagemakerEndpoint
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import os
import json

from langchain.vectorstores import Pinecone

from langchain.chains import RetrievalQA

from langchain.chains.question_answering import load_qa_chain


from dotenv import load_dotenv
load_dotenv()




from config import *
from src.utils import connect_index

region = os.environ["AWS_REGION"]
endpoint_name = os.environ["SAGEMAKER_ENDPOINT_NAME"]
print(endpoint_name)

/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Falcon7b


In [3]:
embeddings = CohereEmbeddings(model=COHERE_MODEL_NAME, cohere_api_key=COHERE_API_KEY)

In [4]:
index = connect_index(PINECONE_INDEX_NAME)
vectorstore = Pinecone(index, embeddings.embed_query, "text")

2023-08-11 10:24:24.643 
  command:

    streamlit run /Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-08-11 10:24:25.133 Connected to Pinecone index stab-test


In [4]:
parameters = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.1,
    "max_new_tokens": 256,
    "num_return_sequences": 4,
}
    
class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"
        
        def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
            input_str = json.dumps({"inputs": prompt, "parameters": parameters, **model_kwargs})
            return input_str.encode('utf-8')
        
        def transform_output(self, output: bytes) -> str:
            response_json = json.loads(output.read().decode("utf-8"))
            return response_json[0]['generated_text']

In [5]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])



In [6]:
content_handler = ContentHandler()

llm=SagemakerEndpoint(
     endpoint_name=endpoint_name, 
     region_name=region, 
     model_kwargs={"max_new_tokens": 200, "top_p": 0.9, "max_length": None, "temperature":1e-10},
     endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
     content_handler=content_handler,
     credentials_profile_name="fundamentl-ai"
 )

2023-08-11 09:50:35.485 Found credentials in shared credentials file: ~/.aws/credentials


In [40]:
chain = load_qa_chain(prompt=QA_CHAIN_PROMPT, llm=llm)

In [41]:
question = "What is the capital of France?"

In [42]:
from langchain.docstore.document import Document
example_doc_1 = """
Peter and Elizabeth took a taxi to attend the night party in the city. While in the party, Elizabeth collapsed and was rushed to the hospital.
Since she was diagnosed with a brain injury, the doctor told Peter to stay besides her until she gets well.
Therefore, Peter stayed with her at the hospital for 3 days without leaving.
"""

docs = [
    Document(
        page_content=example_doc_1,
    )
]

In [43]:
#qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(), chain_type="stuff", return_source_documents=True, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [44]:
prompt_template = """
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it 
does not know.
{context}
Instruction: Based on the above documents, provide a detailed answer for, {question} 
Answer "I don't know" 
if info is not present in the document. 
Solution:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [45]:
result = qa_chain({"query":"What is rainfall data?"})

In [46]:
result['result'].split('\n')[-1]

'Rainfall data is a type of data that is collected from weather stations or gauges. It is data that is collected on a regular basis and is used to understand the amount of precipitation that has occurred in a particular area. Rainfall data is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to under

In [47]:
result['result'].split('\n')[6]

''

In [48]:
result['source_documents']

[Document(page_content='In order to help correct for the inaccuracies and errors that are present in the two types of data and take advantage of their individual strengths, research institutions have produced rainfall datasets that combine station and satellite/radar observations. Datasets that combine these observations are the recommended source of data for humanitarian purposes.\n\nObservational precipitation data can be presented in various formats, each providing unique insights into precipitation patterns. The most common representation is through absolute values, which reflect the actual amount of precipitation observed at a specific location and time, typically measured in millimeters. Precipitation averages, which are calculated from historical records over a period of typically at least 10 years, can be produced for a specific location, season, or year. A precipitation anomaly represents the deviation from the expected or average precipitation over a particular period, which 

In [49]:
result = qa_chain({"query": "What is Twitter?"})

In [52]:
result['result'].split('\n')[-1]

'In summary, Twitter is a social media platform that allows users to post and share short messages with other users. It is free to use and'

In [51]:
result['source_documents']

[Document(page_content="Enguehard, C. 1994. Acquisition of terminology from colloquial texts. In Proceedings, Computational Linguistics for Speech and Handwriting Recognition (CLSHR).\n\nGrefenstette, G. 1994. Explorations in Automatic Thesaurus Discovery. Dordrecht, The Netherlands: Kluwer Academic Publisher.\n\nJacquemin, C. 1994. Recycling terms into a partial parser. In Proceedings, 4th Conference on Applied Natural Language Processing (ANLP'94),  113-118.\n\nLewis, D. D., and Croft, W. B. 1990. Term clustering of syntactic phrasess. In Proceedings, 13th Annual International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR'90),  385-404.\n\nResnik, P. 1993. Selection and Information : A Class-Based Approach to Lexical Relationships. Ph.D. thesis, University of Pennsylvania, Institute for Research in Cognitive Science.\n\nShieber, S. N. 1986. An Introduction to Unification-Based Approaches to Grammar. CSLI Lecture Notes 4. Stanford, CA: CSLI.\n\nSmadj

In [7]:
embeding = embeddings.embed_query("Que es la precipitacion?")

In [8]:
len(embeding)

768

# Sagemaker Embeddings

In [6]:
from langchain.embeddings.sagemaker_endpoint import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from typing import Dict, List, Optional
import json

In [9]:
class ContentHandlerEmbeddings(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json


content_handler_embeddings = ContentHandlerEmbeddings()

In [11]:
stsb = SagemakerEndpointEmbeddings(endpoint_name="stsb", region_name="us-east-1", credentials_profile_name="fundamentl-ai", content_handler=content_handler_embeddings)

In [12]:
query = "What is rainfall data?"

In [13]:
embedding = stsb.embed_documents([query])

In [14]:
embedding

[[[[0.3020949959754944,
    -0.0730985477566719,
    2.943516731262207,
    0.08391545712947845,
    0.03553830459713936,
    1.113262414932251,
    0.5701084733009338,
    -0.2569727897644043,
    -0.4793245494365692,
    0.1689087152481079,
    -0.9623517990112305,
    -0.055724021047353745,
    -0.39826667308807373,
    0.06811810284852982,
    1.0388829708099365,
    -0.30591291189193726,
    0.20456558465957642,
    0.11526253819465637,
    1.3143237829208374,
    -0.0954272598028183,
    -0.3546774983406067,
    0.17505140602588654,
    1.0472424030303955,
    -0.5605553388595581,
    -0.6055812239646912,
    0.15448929369449615,
    1.1074249744415283,
    -0.6820714473724365,
    0.22522518038749695,
    -0.3832176923751831,
    0.24853205680847168,
    0.43508949875831604,
    -0.6169328689575195,
    -0.4087351858615875,
    -0.5474420189857483,
    -0.6849663257598877,
    -0.6995367407798767,
    0.5253391265869141,
    -0.13333046436309814,
    -0.21370890736579895,
    -0

In [89]:
vectorstore_sagemaker = Pinecone(index, stsb.embed_query, "text")

In [91]:
vectorstore_sagemaker.similarity_search_with_score(query)

[(Document(page_content="(wi\n\n\n\n1, wi) and\n\npG,Ti(w|v) the probability estimates based on a given classification G and training corpus Ti. Given a particular Ti, the probability of the ``held-out'' part\n\n(wi\n\n\n\n1, wi) is\n\npG,Ti(wi|wi-1). The probability of the complete corpus, where each pair is in turn considered the ``held-out'' part is the leaving-one-out likelihood LLO\n\nAs shown before,\n\npG,Ti(g2, w)=pG,Ti(w) (if the classification Gis a function) and since\n\npTi(w) is actually independent of G, one can drop it out of the maximization and thus need not specify an estimate for it.\n\nAs will be shown later, one can guarantee that every class g1 and g2 has been seen at least once in the ``retained'' part and one can thus use relative counts as estimates for class uni-grams: p_{G,T_{i}}(g_{1})  =  \\frac{N_{T_{i}}(g_{1})}{N_{T_{i}}}\\\\ p_{G,T_{i}}(g_{2})  =  \\frac{N_{T_{i}}(g_{2})}{N_{T_{i}}}. \\end{eqnarray} -->\n\nClustering Algorithm\n\nWe will now determine th

# Sagemaker embeddings + LLM

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

NameError: name 'llm' is not defined